In [ ]:
def dec2bin(num=0, length=1):
    binChar = bin(num)
    binChar = list(binChar[2:]) # 15 -> '0b1111'
    if len(binChar) < length:
        binChar = ['0' for ii in range(length - len(binChar))] + binChar
    binNum = [int(aa) for aa in binChar]
    return binNum
for i in range(16):
    print dec2bin(i, 9)


In [ ]:
A = range(1, 10)
B = range(11, 30)

newA = A[:5]+B[5:]
newB = B[:5]+A[5:]
print newA, newB

In [ ]:
import re
a = ['b0_Bp1', 'b1_Bp2', 'sigma1_Bp1', 'sigma2_MGS_dev', 'b0Bp2', 'sigma2D_MGS_dev']
for aa in a:
    if re.match(r'b[0-9]*[-_AaBbPp0-9]*', aa):
        print aa
    if (re.match(r'sigma[\S]*', aa)) and (not re.match(r'sigma2D[\S]*', aa)):
        print 2, aa

In [ ]:
a, b = ('a', 'b')
print a, b

In [ ]:
import random
import math
import copy
import re
import pandas as pd
import os, os.path

def gen_term_pairs(selectedTerms, parmNames):
    # return a table of search param name and (sigmaVal, truncation level), e.g., 'Ap': sigmaAp, b0_ratio
    termpairs = {}
    noneSigmaTerm = {"A", "Mav", "Slope"}

    sigmaParms = [aa for aa in parmNames if re.match(r'sigma[\S]*', aa) and (not re.match(r'sigma2D[\S]*', aa))]
    truncRatioParms = [aa for aa in parmNames if re.match(r'b[0-9]*[-_AaBbPp0-9]*', aa)]

    fixedTermPairs = {"Ap": "b0_ratio", "Bp": "b0_ratio", "Am": "b0m_ratio", "Bn": "b0n_ratio"}

    for curTerm in selectedTerms:
        curSigma = ""
        curTruncRatio = ""
        if curTerm not in noneSigmaTerm:
            if curTerm in fixedTermPairs.keys():
                curSigma = r'sigma{}'.format(curTerm)
                curTruncRatio = fixedTermPairs[curTerm]
            else:
                for sigmaName in sigmaParms:
                    if curTerm in sigmaName:
                        curSigma = sigmaName
                        break
                for truncRatio in truncRatioParms:
                    if curTerm in truncRatio:
                        curTruncRatio = truncRatio
                        break
        termpairs[curTerm] = (curSigma, curTruncRatio)
    return termpairs

def covert_job_parms(termpairs, jobparms):
    '''
    termpairs: all the resist terms and its components, PRS is not supported yet
    jobparms: table of search param name and value
    return a table of search param name and (sigmaVal, truncation level), e.g., 'Ap': 30, 1.2
    '''
    jobinputs = {}
    for curTerm, val in termpairs.items():
        curSigmaName, curTruncRatioName = val
        curSigmaVal = 0.
        curTruncRatioVal = 0.
        if curSigmaName != "":
            assert(curSigmaName in jobparms.keys())
            curSigmaVal = jobparms[curSigmaName]
        if curTruncRatioName != "":
            assert(curTruncRatioName in jobparms.keys())
            curTruncRatioVal = jobparms[curTruncRatioName]
        jobinputs[curTerm] = (curSigmaVal, curTruncRatioVal)
    return jobinputs

if __name__ == "__main__":
    cwd = os.getcwd()
    infile = os.path.join(cwd, "GA_init_setting.xlsx")
    df = pd.read_excel(infile)

    parmMin = df[['parmName', 'parmMin']].set_index('parmName').to_dict().values()[0]
    parmMax = df[['parmName', 'parmMax']].set_index('parmName').to_dict().values()[0]

    useterms = ['AG1', 'AG2', 'Ap', 'Bp', 'Am', 'Bn', 'MG1', 'Slope']
    termpairs = gen_term_pairs(useterms, parmMin)
    for jobparms in [parmMin, parmMax]:
        print covert_job_parms(termpairs, jobparms)

In [ ]:
# linspace samples
import numpy as np
a=np.linspace(0.2, 2.4, 32)
print a, len(a)
print np.linspace(0.2, 2.4, 5)
print np.linspace(0.2, 0.69677419, 8)
print np.linspace(1.90322581, 2.4, 8)
print "\n"

print np.linspace(0.2, 2.4, 5)
print np.linspace(0, 20, 32)
print np.linspace(50, 200, 64)

In [ ]:
import random
print random.randint(0, 99)

In [ ]:
import numpy as np
a =  range(10)
acculumatedLen = lambda x,y: np.sum(x[0:y+1])
for ii, vv in enumerate(a):
    print acculumatedLen(a, ii)

In [ ]:
# test weighted random choice
## test linear ranking selection
import numpy as np
from numpy import random
# draw = choice(list_of_candidates, number_of_items_to_pick, p=probability_distribution)
random.seed(0)
rms = [round(random.random(), 3) for i in range(10)]
print rms
ascendingIndice  = np.argsort(rms)
descendingIndice = ascendingIndice[::-1]
print ascendingIndice
print descendingIndice
singleNum = len(rms)
sp = 1.5 # 1<sp<2, selection pressure, prob of selecting best compared to average
fitness = lambda x: 2 - sp + 2.*(sp-1)*x/(singleNum-1)

props = [0. for ii in range(singleNum)]
for ii, jj in enumerate(descendingIndice):
    print jj, fitness(ii)
    props[jj] = fitness(ii) # the probability in loc jj is by a portion of ii 
sumFitness = sum(props)
props = [1.*x/sumFitness for x in props]

# propDistrubution = [fitness(ii) for ii in descendingIndice]

# print propDistrubution

random.seed(0)
draw = random.choice(rms, 5, p=props, replace=False)
print draw
random.seed(0)
draw = choice(range(singleNum), 5, p=props, replace=False)
print draw

In [17]:
## test recursive tournament selection
import numpy as np
from numpy import random
random.seed(0)
rmses = [round(random.random(), 3) for i in range(10)]
singleNum = len(rmses)
concatList = lambda(x, y): x+y[:]

def tournament(rmses, singlepool, tagetSize, tourSize = 2):
    assert(tagetSize <= len(singlepool))
    if tagetSize == 0:
        return []
    if len(singlepool) < tourSize:
        return singlepool
    draw = random.choice(singlepool, tourSize, replace=False)
#     print 'Singlepool: %s, single rms: %s\n' % (str(singlepool), str([rmses[ii] for ii in singlepool]))
    
    drawRmses = [rmses[ii] for ii in draw]
    winneNo = np.argsort(drawRmses)[0]
    winner = draw[winneNo]
#     print 'draw: %s, draw rms: %s, winner: %s\n' % (str(draw), str(drawRmses), str(winner))
    
    # append winnerIdx into matepool, and remove it from singlepool
    tagetSize -= 1
    
#     print 'matepool: ' + str(matepool)
    try:
        winnerIdx = singlepool.index(winner)
        del singlepool[winnerIdx] 
    except ValueError:
        raise ValueError('The winner {} is not in pre-defined list: {}'.fromat(winner, singlepool))
    return [winner]+tournament(rmses, singlepool, tagetSize, tourSize)[:]    

for i in range(12):
    random.seed(0)
    print tournament(rmses, range(singleNum), i, 2)

[]
[2]
[2, 4]
[2, 4, 3]
[2, 4, 3, 5]
[2, 4, 3, 5, 1]
[2, 4, 3, 5, 1, 9]
[2, 4, 3, 5, 1, 9, 7]
[2, 4, 3, 5, 1, 9, 7, 0]
[2, 4, 3, 5, 1, 9, 7, 0, 6]
[2, 4, 3, 5, 1, 9, 7, 0, 6, 8]


AssertionError: 

In [9]:
x = [0.383, 0.424, 0.438, 0.545, 0.549, 0.603, 0.646, 0.715, 0.892, 0.964]
y = [ 0.646,  0.892,  0.438,  0.646,  0.545]
z = []
w = y+z[:]
print w

[0.646, 0.892, 0.438, 0.646, 0.545]
